In [1]:
import cv2
import numpy as np
from numpy.core.numeric import True_
from numpy.lib.arraysetops import unique
import pandas as pd
# import glob
import os
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import datetime
import matplotlib.pyplot as plt
import yaml
import shutil
import json

from video_utils_virat import VideoUtils

%matplotlib inline

## Dataset Selection

In [2]:

# setup
dataset_dir_path = './datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_000008'
video_name_orig = video_name + video_ext
video_dest_path = './' + video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


video_name_new = 'ann_yml_'

annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


video_name_new = video_name_new + video_name + '.avi'#video_ext

In [3]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)


if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df


In [4]:
# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

Loading annotations...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,0.0,0.0,0.000000,368,659,426,799,Person
1,0.0,1.0,0.033333,367,658,426,799,Person
2,0.0,2.0,0.066667,367,658,426,799,Person
3,0.0,3.0,0.100000,367,658,426,799,Person
4,0.0,4.0,0.133333,367,658,426,799,Person


# Add velocities

In [5]:
# Add velocities for the video

vidcap = cv2.VideoCapture(video_src_path + video_name_orig)
fps = 30
scale = 1/15
if vidcap.isOpened():
    fps = vidcap.get(cv2.CAP_PROP_FPS)  
    # scale = #TBD


def find_mid(a, b):
  c = (a.astype(float) + b.astype(float))/2
  return c

def find_delta(x):
  dx = x.diff()
  dx[0] = np.nan
  dx.fillna(method='backfill', inplace=True)
  return dx

def find_vel(vx,vy):

  vel = np.sqrt(vx * vx + vy * vy).round(2)
  return vel#, vx, vy

for id in df_bbox['object_id'].unique():
  mask = (df_bbox['object_id']==id)
  sub_df = df_bbox[mask]
  x  = find_mid(sub_df['bb_top'], sub_df['bb_bottom'])
  y  = find_mid(sub_df['bb_left'], sub_df['bb_right'])
  dx = find_delta(x)
  dy = find_delta(y)
  vx = (dx * fps * scale)
  vy = (dy * fps * scale)

  df_bbox.loc[mask, 'x']  = x
  df_bbox.loc[mask, 'y']  = y
  df_bbox.loc[mask, 'dx'] = dx
  df_bbox.loc[mask, 'dy'] = dy
  df_bbox.loc[mask, 'vx'] = (dx * fps * scale)
  df_bbox.loc[mask, 'vy'] = (dy * fps * scale)
  df_bbox.loc[mask, 'vel']  = np.sqrt(vx * vx + vy * vy).round(2)

df_bbox.head()

,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category,x,y,dx,dy,vx,vy,vel
0,0.0,0.0,0.000000,368,659,426,799,Person,729.0,397.0,-0.5,-0.5,-0.999001,-0.999001,1.41
1,0.0,1.0,0.033333,367,658,426,799,Person,728.5,396.5,-0.5,-0.5,-0.999001,-0.999001,1.41
2,0.0,2.0,0.066667,367,658,426,799,Person,728.5,396.5,0.0,0.0,0.000000,0.000000,0.00
3,0.0,3.0,0.100000,367,658,426,799,Person,728.5,396.5,0.0,0.0,0.000000,0.000000,0.00
4,0.0,4.0,0.133333,367,658,426,799,Person,728.5,396.5,0.0,0.0,0.000000,0.000000,0.00


## Annotate Video

In [6]:
from video_utils_virat import VideoUtils
from video_utils_virat import DirectoryUtils
# vUtils = VideoUtils(categoriesDict) 
# types_lst = df_bbox['category'].unique()
# types_dict = {}
# for i in range(len(types_lst)):
#   types_dict[types_lst[i]] = i

# vUtils = VideoUtils(types_dict) 
# drUtils = DirectoryUtils()

# start_time =0
# vUtils.AnnotateVideo(video_dest_path, video_src_path + video_name_orig, video_name_new, df_bbox, start_time, duration=[], save_images=False)

# drUtils.ClearFileType(video_dest_path,".jpg")
# shutil.move(video_src_path + video_name_orig, video_src_path + 'UsedVideos/' + video_name_orig)

In [9]:
# Load Yolo
yolo_weight_file = "./darknet/yolov3.weights"
yolo_cfg_file = "./darknet/cfg/yolov3.cfg"
yolo_names_file = "./darknet/data/coco.names"

yolo = cv2.dnn.readNet(yolo_weight_file, yolo_cfg_file)
classes = []
with open(yolo_names_file, "r") as f:
    classes = f.read().splitlines()
layer_names = yolo.getLayerNames()
len(classes)

classes_dict = {}
for i in range(len(classes)):
  classes_dict[classes[i]] = i

vUtils = VideoUtils(classes_dict) 
start_time = 0
video_duration=None
vUtils.YoloOnVideo(yolo, video_dest_path, video_src_path + video_name_orig, video_name_new, start_time_sec=start_time, duration_sec=video_duration, save_images=False)



Total frames in video: 5805.00 @ 29.97 frames/sec
Created frame id  0, 0.00 sec in video; completed:  0.0 %
Created frame id 25, 0.83 sec in video; completed:  16.8 %
Created frame id 50, 1.67 sec in video; completed:  33.6 %
Created frame id 75, 2.50 sec in video; completed:  50.3 %
Created frame id 100, 3.34 sec in video; completed:  67.1 %
Created frame id 125, 4.17 sec in video; completed:  83.9 %
Done: Created video: ./VIRAT_S_000008/0-4_ann_yml_VIRAT_S_000008.avi


In [8]:
from video_utils_virat import VideoUtils
vUtils = VideoUtils() 
vUtils.ChangeVideoFrameRate('video_20190906150731_3.avi','test2.avi', 10, dir='./')

TypeError: ChangeVideoFrameRate() got an unexpected keyword argument 'dir'

In [ ]:
from video_utils_virat import VideoUtils
vUtils = VideoUtils() 
vUtils.YoloTheWebcam()